# **AI Capitalism-1**

We get the data from reddit using PRAW. That data is added to a global variable threads, which we later clean, remove stop words, lemmatize. This data is then sent for sentiment analysis to NLTK vader sentiment Intensity analyser, transformer's sentiment analysis pipeline and a BERT model(specifics are below). We indivudually perform keyword extraction on all the data using ocunt vectorizer and TFIDF vectorizer, use logistic regression to fit tranform these vectors and get unigram, bigram, trigram features.


To run this code, either use your reddit accounts' client key and client secret to access reddit posts or refer to secret.txt for my credentials. Replace these credentials in client_id="client_id", client_secret="client_secret" to access Reddit API.

Run all the cells one after the other to see how our pipeline works. 

You can also substitute Employment Law for other subreddits like "walmart", "AmazonFC", "Target", "HomeDepot", "UPSers", "Fedexers", "kroger", "starbucks", "DairyQueen", "marriott", "WalgreensStores", "TjMaxx", "costco", "McDonaldsEmployees" to analyze company specific toxicity or use generic subreddits like "employeesonly", "EmploymentLaw", "Employment", "EmploymentAssistance", etc

Bonus: By just swapping keywords for subreddit thread, this same exact model can be used for a wide variety of things like finding opions about Colleges, Schools, Movies, Sports or anything else!!!


## Installing required dependencies


In [ ]:
pip install praw


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.0/191.0 kB 3.1 MB/s eta 0:00:00


In [ ]:
pip install nltk


In [ ]:
pip install transformers

In [ ]:
pip install happytransformer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.8/252.8 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.1 MB/s eta 0:00:00


## Importing required imports

In [ ]:
from pprint import pprint
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import praw
import re
import string
import warnings
warnings.filterwarnings('ignore')

import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize

nltk.download('vader_lexicon')
nltk.download("stopwords")
nltk.download('punkt')
nltk.download('wordnet')

from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from prettytable import PrettyTable
from transformers import pipeline
from happytransformer import HappyTextClassification

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!



## Getting data from Reddit API using PRAW


In [ ]:
# Using personal credentials of Reddit account to get the data
user_agent = "Scraper 1.0 by ME"
reddit = praw.Reddit(
    client_id="client_id",
    client_secret="client_secret",
    user_agent=user_agent
)

In [ ]:
# For now we are scanning single subreddit which is "EmploymentLaw", we can change it or add new SubReddits.
sub_reddit = 'EmploymentLaw'
threads = []
for submission in reddit.subreddit(sub_reddit).hot(limit=None):
  thread = submission.title+" "+submission.selftext
  threads.append(thread)



In [ ]:

# Initially we added comments also to data, but observed that comments were positive to a negative post, which caused ambiguity
# for submission in reddit.subreddit(sub_reddit).hot(limit=None):

  # pprint.pprint(vars(submission))
  # thread = submission.title+" "+submission.selftext
  # submission.comments.replace_more(limit=None)
  # for comment in submission.comments.list():
  #     # print(comment.body)
  #     thread+=" "+comment.body
  # threads.append(thread)


In [ ]:
len(threads)

998

In [ ]:
df=pd.DataFrame(threads)
df.head()


,0
0,Effective Immediately: Rules
1,Company car ordered but I want to leave - Engl...
2,Would I violate my non-compete if “competitive...
3,Ontario (Can)/US company located in North Caro...
4,"California - boss pressured me to do drugs, di..."


In [ ]:
df.to_csv('Raw_Reddit_Data.csv', header=False, encoding='utf-8',index=False)

## Sentiment Analysis using NLTK


In [ ]:
# Performing sentiment analysis on reddit data
sentiment_intensity_analyser=SIA()
results_NLTK=[]

for t in threads:
  t_score = sentiment_intensity_analyser.polarity_scores(t)
  t_score['thread'] = t
  results_NLTK.append(t_score)

print(results_NLTK[:3])

[{'neg': 0.0, 'neu': 0.392, 'pos': 0.608, 'compound': 0.4767, 'thread': 'Effective Immediately: Rules '}, {'neg': 0.133, 'neu': 0.718, 'pos': 0.149, 'compound': 0.0387, 'thread': 'Company car ordered but I want to leave - England UK '}, {'neg': 0.039, 'neu': 0.881, 'pos': 0.08, 'compound': 0.9626, 'thread': 'Would I violate my non-compete if “competitive services” are defined as marketing as a whole and I provide SEO services? (Philadelphia PA) So basically just to give you the gist of my situation, I was terminated back in November for violating my contract. To give you some background info, I worked as an SEO associate at a large marketing agency that offered plenty of other marketing services (not just SEO). This was in Pennsylvania\n\nI won\'t go too much into the background, but during my employment, I tried my best to bring in leads so I could earn a referral %. Their minimum budget was $5,000, so I could consult with smaller clients who are looking at a $500-750 budget. I though

In [ ]:
df_sentient_1=pd.DataFrame.from_records(results_NLTK)
df_sentient_1.head()

,neg,neu,pos,compound,thread
0,0.000,0.392,0.608,0.4767,Effective Immediately: Rules
1,0.133,0.718,0.149,0.0387,Company car ordered but I want to leave - Engl...
2,0.039,0.881,0.080,0.9626,Would I violate my non-compete if “competitive...
3,0.066,0.784,0.150,0.9868,Ontario (Can)/US company located in North Caro...
4,0.167,0.715,0.118,-0.8630,"California - boss pressured me to do drugs, di..."


In [ ]:
df_sentient_1['label']=0
df_sentient_1.loc[df_sentient_1['compound'] > 0.1,'label'] = 1
df_sentient_1.loc[df_sentient_1['compound'] < -0.1, 'label'] = -1
df_sentient_1.head()

,neg,neu,pos,compound,thread,label
0,0.000,0.392,0.608,0.4767,Effective Immediately: Rules,1
1,0.133,0.718,0.149,0.0387,Company car ordered but I want to leave - Engl...,0
2,0.039,0.881,0.080,0.9626,Would I violate my non-compete if “competitive...,1
3,0.066,0.784,0.150,0.9868,Ontario (Can)/US company located in North Caro...,1
4,0.167,0.715,0.118,-0.8630,"California - boss pressured me to do drugs, di...",-1


In [ ]:
# Only pick certain columsn, drop remaining for further analysis.
df_sentiment_export_1=df_sentient_1[['thread','label']]
df_sentiment_export_1.head()

,thread,label
0,Effective Immediately: Rules,1
1,Company car ordered but I want to leave - Engl...,0
2,Would I violate my non-compete if “competitive...,1
3,Ontario (Can)/US company located in North Caro...,1
4,"California - boss pressured me to do drugs, di...",-1


In [ ]:
# Storing file in csv for further analysis
df_sentiment_export_1.to_csv('sentiment_NLTK.csv',encoding='utf-8',index=False)

In [ ]:
df_sentiment_export_1.label.value_counts()

 1    478
-1    462
 0     58
Name: label, dtype: int64

## Feature identification based on sentiment data by NLTK


In [ ]:
stop_words = set(stopwords.words("english"))

In [ ]:
# Initialize the WordNet Lemmatizer
wordnet = WordNetLemmatizer()
def text_preproc(x):
  # Convert text to lowercase to maintain consistency
  x = x.lower()
  
  # Remove stop words from the text  
  x = ' '.join([word for word in x.split(' ') if word not in stop_words])
  
  # Encode to ASCII to remove non-ASCII characters, then decode back to string
  x = x.encode('ascii', 'ignore').decode()
  
  # Remove URLs, replacing them with space
  x = re.sub(r'https*\S+', ' ', x)
  
  # Remove all mentions (words starting with '@'), replacing them with space
  x = re.sub(r'@\S+', ' ', x)

  # Remove hashtags (words starting with '#'), replacing them with space
  x = re.sub(r'#\S+', ' ', x)

  # Remove contractions (words starting with an apostrophe)
  x = re.sub(r'\'\w+', '', x)
  
  # Replace all punctuations with space
  x = re.sub('[%s]' % re.escape(string.punctuation), ' ', x)
  
  # Remove words containing numbers
  x = re.sub(r'\w*\d+\w*', '', x)

  # Replace multiple spaces with a single space
  x = re.sub(r'\s{2,}', ' ', x)

  # Return the processed text
  return x

In [ ]:
print(df_sentiment_export_1.columns)

Index(['thread', 'label'], dtype='object')


In [ ]:
# Add a new column into same dataframe after cleaning the text
df_sentiment_export_1['clean_text'] = df_sentiment_export_1.thread.apply(text_preproc)

In [ ]:
df_sentiment_export_1.head(10)

,thread,label,clean_text
0,Effective Immediately: Rules,1,effective immediately rules
1,Company car ordered but I want to leave - Engl...,0,company car ordered want leave england uk
2,Would I violate my non-compete if “competitive...,1,would violate non compete competitive services...
3,Ontario (Can)/US company located in North Caro...,1,ontario can us company located north carolina ...
4,"California - boss pressured me to do drugs, di...",-1,california boss pressured drugs drugs company ...
5,Forced lunches in Missouri? Missouri is a no b...,-1,forced lunches missouri missouri break state e...
6,Georgia - Regular shift is 11 hours. Being tol...,-1,georgia regular shift hours told pto hours als...
7,Question: intentionally not adhering to contra...,1,question intentionally adhering contract cap t...
8,Mom's boss stole her car? Mom's boss stole her...,-1,mom boss stole car mom boss stole car hi every...
9,I’m in South Carolina my job make me pay for b...,-1,im south carolina job make pay breaking bottle...


In [ ]:
# Counting most common 100 words, just for an idea (not used further in the model)
Counter(" ".join(df_sentiment_export_1['clean_text']).split()).most_common(100)

[('i', 1251),
 ('work', 1166),
 ('company', 944),
 ('time', 869),
 ('would', 765),
 ('pay', 686),
 ('job', 684),
 ('employer', 593),
 ('hours', 552),
 ('get', 508),
 ('employee', 471),
 ('told', 465),
 ('im', 413),
 ('week', 412),
 ('one', 409),
 ('paid', 401),
 ('working', 400),
 ('manager', 396),
 ('new', 391),
 ('day', 388),
 ('days', 368),
 ('also', 366),
 ('know', 355),
 ('said', 346),
 ('leave', 342),
 ('employment', 340),
 ('months', 336),
 ('back', 328),
 ('it', 319),
 ('employees', 318),
 ('contract', 316),
 ('hr', 315),
 ('even', 310),
 ('like', 292),
 ('boss', 286),
 ('year', 286),
 ('years', 275),
 ('legal', 272),
 ('me', 266),
 ('weeks', 264),
 ('go', 259),
 ('this', 258),
 ('last', 249),
 ('could', 245),
 ('never', 244),
 ('since', 243),
 ('want', 242),
 ('due', 241),
 ('take', 239),
 ('us', 237),
 ('asked', 235),
 ('position', 234),
 ('got', 227),
 ('still', 222),
 ('need', 217),
 ('going', 214),
 ('make', 211),
 ('dont', 211),
 ('worked', 210),
 ('anything', 208),
 ('tw

In [ ]:
# Define a custom tokenizer class
class LemmaTokenizer(object):
  # Constructor for initializing the WordNet Lemmatizer
  def __init__(self):
    self.wordnetlemma = WordNetLemmatizer()
 
  # Callable method for the tokenizer
  def __call__(self,text):
    # Tokenize the input text and then lemmatize each token/word
    # 'word_tokenize' splits the text into individual words/tokens
    # 'lemmatize' method converts each word to its base or root form
    return [self.wordnetlemma.lemmatize(word) for word in word_tokenize(text)]

In [ ]:
# Split data into train, test in 70 : 30 ratio
train_NLTK,test_NLTK=train_test_split(df_sentiment_export_1,test_size=.3,random_state=42,shuffle=True)

print(len(train_NLTK))
print(len(test_NLTK))
print(train_NLTK.columns)

698
300
Index(['thread', 'label', 'clean_text'], dtype='object')


In [ ]:
#checking for unigram word frequencies using two approaches count vector and TF-IDF

# Initialize a countvectorizer with custome settings
countvect_NLTK = CountVectorizer(analyzer = "word",tokenizer = LemmaTokenizer(), ngram_range=(1,1),min_df=10,max_features=500)

# Initialize a tfidfvectorizer with same custome settings
tfidfvect_NLTK = TfidfVectorizer(analyzer = "word",tokenizer = LemmaTokenizer(), ngram_range=(1,1),min_df=10,max_features=500)

# Transform the previously cleaned text of training set into matrix of token counts
x_train_count_NLTK = countvect_NLTK.fit_transform(train_NLTK['clean_text']).toarray()

# Transform the previously cleaned text of testing set into matrix of token counts
x_test_count_NLTK = countvect_NLTK.transform(test_NLTK['clean_text']).toarray()

# Transform the previously cleaned text of training set into matrix of token counts
x_train_tfidf_NLTK= tfidfvect_NLTK.fit_transform(train_NLTK['clean_text']).toarray()

# Transform the previously cleaned text of testing set into matrix of token counts
x_test_tfidf_NLTK = tfidfvect_NLTK.transform(test_NLTK['clean_text']).toarray()

# Extract the target labels for the training set
y_train_NLTK = train_NLTK['label']

# Extract the target labels for the tes set
y_test_NLTK = test_NLTK['label']

In [ ]:
lgr=LogisticRegression()

In [ ]:
# Fit the logistic regression model using the training data
lgr.fit(x_train_count_NLTK,y_train_NLTK)

# Evaluate and print the accuracy of the model on the test data
lgr.score(x_test_count_NLTK,y_test_NLTK)

# Access the first set of coefficients (importance of features) from the logistic regression model
lgr.coef_[0]

# Create a PrettyTable to display important features and their corresponding scores
important_feature_unigram_NLTK = PrettyTable(["Feature","Score"])

# Add important features to list to be able to save, display
for feature, importance in zip(countvect_NLTK.get_feature_names_out(),lgr.coef_[0]):
    important_feature_unigram_NLTK.add_row([feature,importance])
important_feature_unigram_NLTK.sortby = "Score"

print(important_feature_unigram_NLTK)

+----------------+------------------------+
|    Feature     |         Score          |
+----------------+------------------------+
|    promoted    |   -1.121579573252094   |
|     thanks     |  -1.0435100366156478   |
|   agreement    |  -0.8324901360919129   |
|    located     |  -0.8002994636096534   |
|       le       |  -0.6912799236421138   |
|   contractor   |   -0.637200261522207   |
|     detail     |  -0.6330499100196275   |
|    benefit     |  -0.6092506076791456   |
|    included    |  -0.6079766678409778   |
|   california   |  -0.5871763874814091   |
|     friend     |  -0.5665596309952142   |
|     later      |  -0.5549533518775472   |
|      area      |  -0.5539631226460631   |
|     worth      |  -0.5490674590321375   |
|     exempt     |  -0.5410589699703481   |
|      best      |  -0.5242766642578617   |
|      post      |  -0.5211740087335442   |
|     unpaid     |  -0.5205901859695834   |
|     still      |  -0.5203483328732629   |
|      wife      |  -0.510833255

In [ ]:
# Code from this point on is same as above and nothing new, so will not be commenting!
lgr=LogisticRegression()
lgr.fit(x_train_tfidf_NLTK,y_train_NLTK)
lgr.score(x_test_tfidf_NLTK,y_test_NLTK)
lgr.coef_[0]

important_feature_TF_IDF_unigram_NLTK = PrettyTable(["Feature","Score"])
for feature, importance in zip(tfidfvect_NLTK.get_feature_names_out(),lgr.coef_[0]):
    important_feature_TF_IDF_unigram_NLTK.add_row([feature,importance])
important_feature_TF_IDF_unigram_NLTK.sortby = "Score"

print(important_feature_TF_IDF_unigram_NLTK)

+----------------+------------------------+
|    Feature     |         Score          |
+----------------+------------------------+
|   agreement    |  -1.1809319605686117   |
|     thanks     |  -0.9737418918987605   |
|     offer      |   -0.89434390703728    |
|   contractor   |  -0.8688861093696623   |
|     friend     |  -0.8389818392419662   |
|    holiday     |  -0.8380509222340183   |
|   california   |  -0.8080578174433893   |
|     bonus      |   -0.788092665276936   |
|      new       |  -0.7687199431684095   |
|    promoted    |  -0.7277883643477189   |
|    overtime    |  -0.7016832830639819   |
|    required    |  -0.6803763611738434   |
|     unpaid     |  -0.6688512060250694   |
|     period     |  -0.6494915992162102   |
|      week      |  -0.6326212746773241   |
|  independent   |  -0.6101819151378183   |
|     salary     |  -0.6087706526484683   |
|    legally     |  -0.5922463896912212   |
|      help      |  -0.5742608632419299   |
|      home      |  -0.572555330

In [ ]:
# Save responses for further analysis
# The major reason to save data into files is because, I used google colab for development and I did not want to lose data when runtime got disconnected.
with open('count_unigrams_NLTK.csv', 'w', newline='') as f_output:
    f_output.write(important_feature_unigram_NLTK.get_csv_string())
with open('tfidf_unigrams_NLTK.csv', 'w', newline='') as f_output:
    f_output.write(important_feature_TF_IDF_unigram_NLTK.get_csv_string())


In [ ]:
# bigram

# The same process mentioned above repeats for bigrams
countvect_2_NLTK = CountVectorizer(analyzer = "word",tokenizer = LemmaTokenizer(), ngram_range=(2,2),min_df=10,max_features=500)
tfidfvect_2_NLTK = TfidfVectorizer(analyzer = "word",tokenizer = LemmaTokenizer(), ngram_range=(2,2),min_df=10,max_features=500)
x_train_count_2_NLTK=countvect_2_NLTK.fit_transform(train_NLTK['clean_text']).toarray()
x_test_count_2_NLTK = countvect_2_NLTK.transform(test_NLTK['clean_text']).toarray()
x_train_tfidf_2_NLTK= tfidfvect_2_NLTK.fit_transform(train_NLTK['clean_text']).toarray()
x_test_tfidf_2_NLTK = tfidfvect_2_NLTK.transform(test_NLTK['clean_text']).toarray()
y_train_2_NLTK = train_NLTK['label']
y_test_2_NLTK = test_NLTK['label']

In [ ]:
lgr=LogisticRegression()
lgr.fit(x_train_count_2_NLTK,y_train_2_NLTK)
lgr.score(x_test_count_2_NLTK,y_test_2_NLTK)
lgr.coef_[0]
i=0
importantfeature_count_2_NLTK = PrettyTable(["Feature","Score"])
for feature, importance in zip(countvect_2_NLTK.get_feature_names_out(),lgr.coef_[0]):
    importantfeature_count_2_NLTK.add_row([feature,importance])
importantfeature_count_2_NLTK.sortby = "Score"

print(importantfeature_count_2_NLTK)

+------------------------+------------------------+
|        Feature         |         Score          |
+------------------------+------------------------+
|     severance pay      |   -1.096564943102478   |
| independent contractor |  -0.9182639998685881   |
|      hour worked       |  -0.7792791353033998   |
|      manager said      |  -0.7561066491658802   |
|        is legal        |   -0.721070808807904   |
|      story short       |  -0.6856318533770631   |
|      current job       |  -0.6632456848407218   |
|       any advice       |  -0.5803274444078178   |
|       make sure        |  -0.5755484522389763   |
|       time work        |  -0.5653612260285288   |
|       first day        |  -0.5564643284668893   |
|        one week        |  -0.49984591438615983  |
|     thanks advance     |  -0.49348824330463303  |
|     work remotely      |  -0.4672182048000778   |
|      anyone else       |  -0.4636864284005596   |
|  greatly appreciated   |  -0.4433418916648113   |
|       woul

In [ ]:
lgr=LogisticRegression()
lgr.fit(x_train_tfidf_2_NLTK,y_train_2_NLTK)
lgr.score(x_test_tfidf_2_NLTK,y_test_2_NLTK)
lgr.coef_[0]

importantfeature_tfidf_2_NLTK = PrettyTable(["Feature","Score"])
for feature, importance in zip(tfidfvect_2_NLTK.get_feature_names_out(),lgr.coef_[0]):
    importantfeature_tfidf_2_NLTK.add_row([feature,importance])
importantfeature_tfidf_2_NLTK.sortby = "Score"

print(importantfeature_tfidf_2_NLTK)

+------------------------+------------------------+
|        Feature         |         Score          |
+------------------------+------------------------+
| independent contractor |  -1.1140297144426379   |
|     severance pay      |  -1.0978913901760694   |
|       make sure        |  -0.8145657374962151   |
|      manager said      |  -0.8130755295677382   |
|        is legal        |  -0.8110861057155507   |
|      hour worked       |  -0.7714255754436365   |
|      story short       |  -0.7603267448597704   |
|       hour week        |  -0.7097028417577802   |
|     thanks advance     |  -0.6795433660741615   |
|       time work        |  -0.6099530487516247   |
|        one week        |  -0.5764962969983466   |
|       last month       |  -0.5656041382184394   |
|        laid off        |   -0.565212037278399   |
|        i worked        |  -0.5639606625786516   |
|       any advice       |  -0.5621332610391272   |
|       start date       |  -0.5575089788243477   |
|       long

In [ ]:
with open('count_biigrams_NLTK.csv', 'w', newline='') as f_output:
    f_output.write(importantfeature_count_2_NLTK.get_csv_string())
with open('tfidf_biigrams_NLTK.csv', 'w', newline='') as f_output:
    f_output.write(importantfeature_tfidf_2_NLTK.get_csv_string())

In [ ]:
# trigram
countvect_3_NLTK = CountVectorizer(analyzer = "word",tokenizer = LemmaTokenizer(), ngram_range=(3,3),min_df=10,max_features=500)
tfidfvect_3_NLTK = TfidfVectorizer(analyzer = "word",tokenizer = LemmaTokenizer(), ngram_range=(3,3),min_df=10,max_features=500)
x_train_count_3_NLTK = countvect_3_NLTK.fit_transform(train_NLTK['clean_text']).toarray()
x_test_count_3_NLTK = countvect_3_NLTK.transform(test_NLTK['clean_text']).toarray()
x_train_tfidf_3_NLTK= tfidfvect_3_NLTK.fit_transform(train_NLTK['clean_text']).toarray()
x_test_tfidf_3_NLTK = tfidfvect_3_NLTK.transform(test_NLTK['clean_text']).toarray()
y_train_3_NLTK = train_NLTK['label']
y_test_3_NLTK = test_NLTK['label']

In [ ]:
lgr=LogisticRegression()
lgr.fit(x_train_count_3_NLTK,y_train_3_NLTK)
lgr.score(x_test_count_3_NLTK,y_test_3_NLTK)
lgr.coef_[0]
i=0
importantfeature_3_NLTK = PrettyTable(["Feature","Score"])
for feature, importance in zip(countvect_3_NLTK.get_feature_names_out(),lgr.coef_[0]):
    importantfeature_3_NLTK.add_row([feature,importance])
importantfeature_3_NLTK.sortby = "Score"

print(importantfeature_3_NLTK)

+------------------+---------------------+
|     Feature      |        Score        |
+------------------+---------------------+
| long story short | -0.7609528867969765 |
|  my question is  |  0.2988103545376654 |
+------------------+---------------------+


In [ ]:
lgr=LogisticRegression()
lgr.fit(x_train_tfidf_3_NLTK,y_train_3_NLTK)
lgr.score(x_test_tfidf_3_NLTK,y_test_3_NLTK)
lgr.coef_[0]

importantfeature_tfidf_3_NLTK = PrettyTable(["Feature","Score"])
for feature, importance in zip(tfidfvect_3_NLTK.get_feature_names_out(),lgr.coef_[0]):
    importantfeature_tfidf_3_NLTK.add_row([feature,importance])
importantfeature_tfidf_3_NLTK.sortby = "Score"

print(importantfeature_tfidf_3_NLTK)

+------------------+---------------------+
|     Feature      |        Score        |
+------------------+---------------------+
| long story short | -0.7558080875117588 |
|  my question is  | 0.30166184215236297 |
+------------------+---------------------+


In [ ]:
with open('count_trigrams_NLTK.csv', 'w', newline='') as f_output:
    f_output.write(importantfeature_3_NLTK.get_csv_string())
with open('tfidf_trigrams_NLTK.csv', 'w', newline='') as f_output:
    f_output.write(importantfeature_tfidf_3_NLTK.get_csv_string())

In [ ]:
# words of lengths 1,2,3
countvect_NLTK_all = CountVectorizer(analyzer = "word",tokenizer = LemmaTokenizer(), ngram_range=(1,3),min_df=10,max_features=1500)
tfidfvect_NLTK_all = TfidfVectorizer(analyzer = "word",tokenizer = LemmaTokenizer(), ngram_range=(1,3),min_df=10,max_features=1500)
x_train_count_NLTK_all = countvect_NLTK_all.fit_transform(train_NLTK['clean_text']).toarray()
x_test_count_NLTK_all = countvect_NLTK_all.transform(test_NLTK['clean_text']).toarray()
x_train_tfidf_NLTK_all= tfidfvect_NLTK_all.fit_transform(train_NLTK['clean_text']).toarray()
x_test_tfidf_NLTK_all = tfidfvect_NLTK_all.transform(test_NLTK['clean_text']).toarray()
y_train_NLTK_all = train_NLTK['label']
y_test_NLTK_all = test_NLTK['label']

In [ ]:
lgr=LogisticRegression()
lgr.fit(x_train_count_NLTK_all,y_train_NLTK_all)
lgr.score(x_test_count_NLTK_all,y_test_NLTK_all)
lgr.coef_[0]
i=0
importantfeature_NLTK_all = PrettyTable(["Feature","Score"])
for feature, importance in zip(countvect_NLTK_all.get_feature_names_out(),lgr.coef_[0]):
    importantfeature_NLTK_all.add_row([feature,importance])
importantfeature_NLTK_all.sortby = "Score"

print(importantfeature_NLTK_all)

+------------------------+------------------------+
|        Feature         |         Score          |
+------------------------+------------------------+
|         thanks         |  -0.7637202493233842   |
|          goal          |  -0.6137023879495236   |
|          free          |  -0.5533279828132339   |
|       agreement        |   -0.526220412410433   |
|         friend         |  -0.49965255589638724  |
|         bonus          |  -0.49393177325035464  |
|        promoted        |  -0.4923402804468471   |
|          rule          |  -0.48653119011205204  |
|        holiday         |  -0.47657597163643434  |
|        located         |  -0.4744054189902033   |
|       effective        |  -0.44652603219701803  |
|       contractor       |  -0.43593965081023484  |
|         match          |  -0.43529737447869826  |
|       california       |  -0.42608925194783837  |
|     severance pay      |  -0.40263786584381195  |
|          best          |  -0.40150994970350445  |
|         pe

In [ ]:
lgr=LogisticRegression()
lgr.fit(x_train_tfidf_NLTK_all,y_train_NLTK_all)
lgr.score(x_test_tfidf_NLTK_all,y_test_NLTK_all)
lgr.coef_[0]

importantfeature2_NLTK_all = PrettyTable(["Feature","Score"])
for feature, importance in zip(tfidfvect_NLTK_all.get_feature_names_out(),lgr.coef_[0]):
    importantfeature2_NLTK_all.add_row([feature,importance])
importantfeature2_NLTK_all.sortby = "Score"

print(importantfeature2_NLTK_all)

+------------------------+------------------------+
|        Feature         |         Score          |
+------------------------+------------------------+
|       agreement        |  -1.0972026849101115   |
|         offer          |  -0.8944591107971198   |
|        holiday         |   -0.840592461429758   |
|         bonus          |  -0.8347250726895503   |
|       contractor       |  -0.8078913463701616   |
|         thanks         |  -0.7948551964552778   |
|         friend         |  -0.6874969302821915   |
|       california       |   -0.677174111832148   |
|          new           |  -0.6748444485007954   |
|        overtime        |  -0.6746088909224174   |
|         salary         |  -0.6644849933801399   |
|          free          |  -0.6611982173875234   |
|         period         |  -0.6159808391188372   |
|          goal          |  -0.6054534811301014   |
|        promoted        |   -0.595441361738792   |
|          rule          |  -0.5581903695876577   |
|          w

In [ ]:
with open('count_NLTK_all.csv', 'w', newline='') as f_output:
    f_output.write(importantfeature_NLTK_all.get_csv_string())
with open('tfidf_NLTK_all.csv', 'w', newline='') as f_output:
    f_output.write(importantfeature2_NLTK_all.get_csv_string())

## Sentiment Analysis using Basic Transformer

In [ ]:
# create pipeline for sentiment analysis
classification = pipeline('sentiment-analysis')

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [ ]:
threads_reddit = df_sentiment_export_1['clean_text'].to_list()

In [ ]:
threads_reddit[:3]

['effective immediately rules ',
 'company car ordered want leave england uk ',
 'would violate non compete competitive services defined marketing whole provide seo services philadelphia pa basically give gist situation terminated back november violating contract give background info worked seo associate large marketing agency offered plenty marketing services not seo pennsylvania i go much background employment tried best bring leads could earn referral minimum budget could consult smaller clients looking budget thought would fair game since employer offers integrated marketing services one service much higher budget requirement so here question comes in violating non compete continue provide seo services i terminated severance agreement states non compete still effect would still apply seo specifically since provide full marketing services seo here sections define competitive products services is competitor competition competitive business competitive products services shall mean fol

In [ ]:
results_basic_transformer=[]

for t in threads_reddit:
  if len(t)<2100:
    tr_score = classification(t)
    tr_score[0].update({'content':t})
    results_basic_transformer.append(tr_score[0])

print(results_basic_transformer[:3])



[{'label': 'POSITIVE', 'score': 0.9992859959602356, 'content': 'effective immediately rules '}, {'label': 'NEGATIVE', 'score': 0.9965292811393738, 'content': 'company car ordered want leave england uk '}, {'label': 'NEGATIVE', 'score': 0.9976819753646851, 'content': 'would violate non compete competitive services defined marketing whole provide seo services philadelphia pa basically give gist situation terminated back november violating contract give background info worked seo associate large marketing agency offered plenty marketing services not seo pennsylvania i go much background employment tried best bring leads could earn referral minimum budget could consult smaller clients looking budget thought would fair game since employer offers integrated marketing services one service much higher budget requirement so here question comes in violating non compete continue provide seo services i terminated severance agreement states non compete still effect would still apply seo specificall

In [ ]:
len(results_basic_transformer)

971

In [ ]:
df_basic_transformer = pd.DataFrame(results_basic_transformer)
df_basic_transformer.head()

,label,score,content
0,POSITIVE,0.999286,effective immediately rules
1,NEGATIVE,0.996529,company car ordered want leave england uk
2,NEGATIVE,0.997682,would violate non compete competitive services...
3,NEGATIVE,0.997027,ontario can us company located north carolina ...
4,NEGATIVE,0.997496,california boss pressured drugs drugs company ...


In [ ]:
df_basic_transformer.count()

label      971
score      971
content    971
dtype: int64

In [ ]:
df_basic_transformer['label_value']=0
df_basic_transformer.loc[df_basic_transformer['label'] == "POSITIVE",'label_value'] = 1
df_basic_transformer.loc[df_basic_transformer['label'] == "NEGATIVE", 'label_value'] = -1
df_basic_transformer.head()

,label,score,content,label_value
0,POSITIVE,0.999286,effective immediately rules,1
1,NEGATIVE,0.996529,company car ordered want leave england uk,-1
2,NEGATIVE,0.997682,would violate non compete competitive services...,-1
3,NEGATIVE,0.997027,ontario can us company located north carolina ...,-1
4,NEGATIVE,0.997496,california boss pressured drugs drugs company ...,-1


In [ ]:
df_basic_transformer.label_value.value_counts()

-1    952
 1     19
Name: label_value, dtype: int64

In [ ]:
df_basic_transformer_stripped=df_basic_transformer[['content','label_value','label']]
df_basic_transformer.count()

label          971
score          971
content        971
label_value    971
dtype: int64

In [ ]:
df_basic_transformer_stripped.to_csv('sentiment_basic_Transformer_with_label.csv',encoding='utf-8',index=False)

In [ ]:
df_basic_transformer_stripped.label.value_counts()

NEGATIVE    952
POSITIVE     19
Name: label, dtype: int64

## Feature Extraction with basic transformer


In [ ]:
train_basic_transformer,test_basic_transformer=train_test_split(df_basic_transformer_stripped,test_size=.3,random_state=42,shuffle=True)

print(len(train_basic_transformer))
print(len(test_basic_transformer))
print(train_basic_transformer.columns)

679
292
Index(['content', 'label_value', 'label'], dtype='object')


In [ ]:
#unigram

countvect_basic_transformer = CountVectorizer(analyzer = "word",tokenizer = LemmaTokenizer(), ngram_range=(1,1),min_df=10,max_features=500)
tfidfvect_basic_transformer = TfidfVectorizer(analyzer = "word",tokenizer = LemmaTokenizer(), ngram_range=(1,1),min_df=10,max_features=500)
x_train_count_basic_transformer=countvect_basic_transformer.fit_transform(train_basic_transformer['content']).toarray()
x_test_count_basic_transformer = countvect_basic_transformer.transform(test_basic_transformer['content']).toarray()
x_train_tfidf_basic_transformer= tfidfvect_basic_transformer.fit_transform(train_basic_transformer['content']).toarray()
x_test_tfidf_basic_transformer = tfidfvect_basic_transformer.transform(test_basic_transformer['content']).toarray()
y_train_basic_transformer = train_basic_transformer['label_value']
y_test_basic_transformer = test_basic_transformer['label_value']
lgr=LogisticRegression()
lgr.fit(x_train_count_basic_transformer,y_train_basic_transformer)
lgr.score(x_test_count_basic_transformer,y_test_basic_transformer)
lgr.coef_[0]
i=0
importantfeature_basic_transformer = PrettyTable(["Feature","Score"])
for feature, importance in zip(countvect_basic_transformer.get_feature_names_out(),lgr.coef_[0]):
    importantfeature_basic_transformer.add_row([feature,importance])
importantfeature_basic_transformer.sortby = "Score"

print(importantfeature_basic_transformer)

+----------------+------------------------+
|    Feature     |         Score          |
+----------------+------------------------+
|       i        |  -0.6982445202725349   |
|     would      |  -0.4913397407962228   |
|   employment   |  -0.4465773013365987   |
|    employee    |  -0.4028863321776362   |
|      said      |  -0.37194953604197106  |
|    manager     |  -0.37006647789179237  |
|      hour      |  -0.3653472554474224   |
|      week      |  -0.36276138955936954  |
|    working     |  -0.3621219226873303   |
|      due       |  -0.35249299177316507  |
|     state      |  -0.34840463287306545  |
|     worker     |  -0.3417517599006436   |
|      pay       |  -0.31748412024981904  |
|     asking     |  -0.3171287006590025   |
|  information   |  -0.3089174673901681   |
|    recently    |  -0.30462666651243997  |
|     claim      |  -0.29926460978914027  |
|      time      |  -0.2980587010123432   |
|      one       |  -0.29326347014916154  |
|     shift      |  -0.292739047

In [ ]:
lgr=LogisticRegression()
lgr.fit(x_train_tfidf_basic_transformer,y_train_basic_transformer)
lgr.score(x_test_tfidf_basic_transformer,y_test_basic_transformer)
lgr.coef_[0]

importantfeature2_basic_transformer = PrettyTable(["Feature","Score"])
for feature, importance in zip(tfidfvect_basic_transformer.get_feature_names_out(),lgr.coef_[0]):
    importantfeature2_basic_transformer.add_row([feature,importance])
importantfeature2_basic_transformer.sortby = "Score"

print(importantfeature2_basic_transformer)

+----------------+------------------------+
|    Feature     |         Score          |
+----------------+------------------------+
|       i        |  -0.5052264155064912   |
|      hour      |  -0.5034134533224666   |
|     would      |  -0.4358663867142656   |
|    employee    |  -0.37337141958140524  |
|      week      |  -0.3677118592053136   |
|    manager     |  -0.34936245566084434  |
|      time      |  -0.2816178255574681   |
|      said      |  -0.27314084638812725  |
|      paid      |  -0.25138527203685884  |
|      told      |  -0.24806467672394655  |
|      one       |  -0.21976677020191995  |
|      due       |  -0.21919978816097369  |
|       me       |  -0.21656582157188295  |
|      sick      |  -0.21563064613863464  |
|      wage      |  -0.21328119177730945  |
|     worker     |  -0.21270165505022234  |
|     offer      |  -0.2116215638913993   |
|     office     |  -0.2099737029427191   |
|    working     |  -0.2099214743367451   |
|    vacation    |  -0.208749216

In [ ]:
with open('count_unigaram_basic_transformer.csv', 'w', newline='') as f_output:
    f_output.write(importantfeature_basic_transformer.get_csv_string())
with open('tfidf_unigram_basic_transformer.csv', 'w', newline='') as f_output:
    f_output.write(importantfeature2_basic_transformer.get_csv_string())

In [ ]:
#bigram

countvect_basic_transformer_2 = CountVectorizer(analyzer = "word",tokenizer = LemmaTokenizer(), ngram_range=(2,2),min_df=10,max_features=500)
tfidfvect_basic_transformer_2 = TfidfVectorizer(analyzer = "word",tokenizer = LemmaTokenizer(), ngram_range=(2,2),min_df=10,max_features=500)
x_train_count_basic_transformer_2=countvect_basic_transformer_2.fit_transform(train_basic_transformer['content']).toarray()
x_test_count_basic_transformer_2 = countvect_basic_transformer_2.transform(test_basic_transformer['content']).toarray()
x_train_tfidf_basic_transformer_2= tfidfvect_basic_transformer_2.fit_transform(train_basic_transformer['content']).toarray()
x_test_tfidf_basic_transformer_2 = tfidfvect_basic_transformer_2.transform(test_basic_transformer['content']).toarray()
y_train_basic_transformer_2 = train_basic_transformer['label_value']
y_test_basic_transformer_2 = test_basic_transformer['label_value']
lgr=LogisticRegression()
lgr.fit(x_train_count_basic_transformer_2,y_train_basic_transformer_2)
lgr.score(x_test_count_basic_transformer_2,y_test_basic_transformer_2)
lgr.coef_[0]
i=0
importantfeature_basic_transformer_2 = PrettyTable(["Feature","Score"])
for feature, importance in zip(countvect_basic_transformer_2.get_feature_names_out(),lgr.coef_[0]):
    importantfeature_basic_transformer_2.add_row([feature,importance])
importantfeature_basic_transformer_2.sortby = "Score"

print(importantfeature_basic_transformer_2)

+------------------------+----------------------+
|        Feature         |        Score         |
+------------------------+----------------------+
|     employment law     | -0.5597628717029692  |
|        can not         | -0.5538047217564354  |
|         let go         | -0.46982220257984847 |
|       feel like        | -0.3931502263010405  |
|        last day        | -0.3911390212202745  |
|        two week        | -0.37817444774931025 |
|       last week        | -0.37145758602441814 |
|        get paid        | -0.36221136620148126 |
|       part time        | -0.35461073684658256 |
|        year ago        | -0.3422961316986528  |
|       told would       | -0.31424851152473077 |
|     vacation time      | -0.31119837174919296 |
|      even though       | -0.3111195452856148  |
|        laid off        | -0.3090902405691372  |
|       sick leave       | -0.30513327245320115 |
|      my question       | -0.3003751571893883  |
|        new job         | -0.29405424289414916 |


In [ ]:
lgr=LogisticRegression()
lgr.fit(x_train_tfidf_basic_transformer_2,y_train_basic_transformer_2)
lgr.score(x_test_tfidf_basic_transformer_2,y_test_basic_transformer_2)
lgr.coef_[0]

importantfeature2_basic_transformer_2 = PrettyTable(["Feature","Score"])
for feature, importance in zip(tfidfvect_basic_transformer_2.get_feature_names_out(),lgr.coef_[0]):
    importantfeature2_basic_transformer_2.add_row([feature,importance])
importantfeature2_basic_transformer_2.sortby = "Score"

print(importantfeature2_basic_transformer_2)

+------------------------+----------------------+
|        Feature         |        Score         |
+------------------------+----------------------+
|     employment law     | -0.31598592677431603 |
|        can not         |  -0.310595291415577  |
|         let go         | -0.28184917524248787 |
|       feel like        | -0.24035834490208102 |
|        get paid        | -0.22426686767648707 |
|       part time        | -0.21048669788241497 |
|        year ago        | -0.20965181568999958 |
|        two week        | -0.2061585696876413  |
|     vacation time      | -0.2016398220402666  |
|      even though       | -0.19965276977547067 |
|       last week        | -0.19675882982219536 |
|  employment contract   | -0.19229662109862364 |
|        last day        | -0.1865879626916507  |
|       last year        | -0.18466110132373936 |
| independent contractor | -0.1808711977362416  |
|      minimum wage      | -0.17999357214849174 |
|         i told         | -0.1792564253020752  |


In [ ]:
with open('count_bigram_basic_transformer.csv', 'w', newline='') as f_output:
    f_output.write(importantfeature_basic_transformer_2.get_csv_string())
with open('tfidf_bigram_basic_transformer.csv', 'w', newline='') as f_output:
    f_output.write(importantfeature2_basic_transformer_2.get_csv_string())

In [ ]:
#trigram

countvect_basic_transformer_3 = CountVectorizer(analyzer = "word",tokenizer = LemmaTokenizer(), ngram_range=(3,3),min_df=10,max_features=500)
tfidfvect_basic_transformer_3 = TfidfVectorizer(analyzer = "word",tokenizer = LemmaTokenizer(), ngram_range=(3,3),min_df=10,max_features=500)
x_train_count_basic_transformer_3=countvect_basic_transformer_3.fit_transform(train_basic_transformer['content']).toarray()
x_test_count_basic_transformer_3 = countvect_basic_transformer_3.transform(test_basic_transformer['content']).toarray()
x_train_tfidf_basic_transformer_3= tfidfvect_basic_transformer_3.fit_transform(train_basic_transformer['content']).toarray()
x_test_tfidf_basic_transformer_3 = tfidfvect_basic_transformer_3.transform(test_basic_transformer['content']).toarray()
y_train_basic_transformer_3 = train_basic_transformer['label_value']
y_test_basic_transformer_3 = test_basic_transformer['label_value']
lgr=LogisticRegression()
lgr.fit(x_train_count_basic_transformer_3,y_train_basic_transformer_3)
lgr.score(x_test_count_basic_transformer_3,y_test_basic_transformer_3)
lgr.coef_[0]
i=0
importantfeature_basic_transformer_3 = PrettyTable(["Feature","Score"])
for feature, importance in zip(countvect_basic_transformer_3.get_feature_names_out(),lgr.coef_[0]):
    importantfeature_basic_transformer_3.add_row([feature,importance])
importantfeature_basic_transformer_3.sortby = "Score"

print(importantfeature_basic_transformer_3)

+------------------+----------------------+
|     Feature      |        Score         |
+------------------+----------------------+
| long story short | -0.21550616828479421 |
|  my question is  | -0.2011424627578996  |
+------------------+----------------------+


In [ ]:
lgr=LogisticRegression()
lgr.fit(x_train_tfidf_basic_transformer_3,y_train_basic_transformer_3)
lgr.score(x_test_tfidf_basic_transformer_3,y_test_basic_transformer_3)
lgr.coef_[0]

importantfeature2_basic_transformer_3 = PrettyTable(["Feature","Score"])
for feature, importance in zip(tfidfvect_basic_transformer_3.get_feature_names_out(),lgr.coef_[0]):
    importantfeature2_basic_transformer_3.add_row([feature,importance])
importantfeature2_basic_transformer_3.sortby = "Score"

print(importantfeature2_basic_transformer_3)

+------------------+----------------------+
|     Feature      |        Score         |
+------------------+----------------------+
| long story short | -0.21310220258160853 |
|  my question is  | -0.19885275808548722 |
+------------------+----------------------+


In [ ]:
with open('count_traigram_basic_transformer.csv', 'w', newline='') as f_output:
    f_output.write(importantfeature_basic_transformer_3.get_csv_string())
with open('tfidf_trigram_basic_transformer.csv', 'w', newline='') as f_output:
    f_output.write(importantfeature2_basic_transformer_3.get_csv_string())

In [ ]:
#all

countvect_basic_transformer_all = CountVectorizer(analyzer = "word",tokenizer = LemmaTokenizer(), ngram_range=(1,3),min_df=10,max_features=500)
tfidfvect_basic_transformer_all = TfidfVectorizer(analyzer = "word",tokenizer = LemmaTokenizer(), ngram_range=(1,3),min_df=10,max_features=500)
x_train_count_basic_transformer_all=countvect_basic_transformer_all.fit_transform(train_basic_transformer['content']).toarray()
x_test_count_basic_transformer_all = countvect_basic_transformer_all.transform(test_basic_transformer['content']).toarray()
x_train_tfidf_basic_transformer_all= tfidfvect_basic_transformer_all.fit_transform(train_basic_transformer['content']).toarray()
x_test_tfidf_basic_transformer_all = tfidfvect_basic_transformer_all.transform(test_basic_transformer['content']).toarray()
y_train_basic_transformer_all = train_basic_transformer['label_value']
y_test_basic_transformer_all = test_basic_transformer['label_value']
lgr=LogisticRegression()
lgr.fit(x_train_count_basic_transformer_all,y_train_basic_transformer_all)
lgr.score(x_test_count_basic_transformer_all,y_test_basic_transformer_all)
lgr.coef_[0]
i=0
importantfeature_basic_transformer_all = PrettyTable(["Feature","Score"])
for feature, importance in zip(countvect_basic_transformer_all.get_feature_names_out(),lgr.coef_[0]):
    importantfeature_basic_transformer_all.add_row([feature,importance])
importantfeature_basic_transformer_all.sortby = "Score"

print(importantfeature_basic_transformer_all)

+------------------------+------------------------+
|        Feature         |         Score          |
+------------------------+------------------------+
|           i            |  -0.7024530046373825   |
|         would          |  -0.5119241261359622   |
|       employment       |  -0.4347335329783244   |
|        employee        |  -0.43283427317529594  |
|          said          |  -0.38730382827511006  |
|          due           |   -0.382598412242826   |
|        manager         |  -0.3818413242117749   |
|          hour          |  -0.3743663726562657   |
|          week          |  -0.3699234552117881   |
|         state          |  -0.3638660126701316   |
|        working         |  -0.35849044496052934  |
|         worker         |  -0.3501802370101665   |
|          time          |  -0.3471991448485691   |
|          pay           |  -0.32817577383107266  |
|         claim          |  -0.3238057385704046   |
|          one           |  -0.3202501092399268   |
|      infor

In [ ]:
lgr=LogisticRegression()
lgr.fit(x_train_tfidf_basic_transformer_all,y_train_basic_transformer_all)
lgr.score(x_test_tfidf_basic_transformer_all,y_test_basic_transformer_all)
lgr.coef_[0]

importantfeature2_basic_transformer_all = PrettyTable(["Feature","Score"])
for feature, importance in zip(tfidfvect_basic_transformer_all.get_feature_names_out(),lgr.coef_[0]):
    importantfeature2_basic_transformer_all.add_row([feature,importance])
importantfeature2_basic_transformer_all.sortby = "Score"

print(importantfeature2_basic_transformer_all)

+------------------------+------------------------+
|        Feature         |         Score          |
+------------------------+------------------------+
|           i            |  -0.5060531665986403   |
|          hour          |  -0.5046733937165042   |
|         would          |  -0.43507803016217195  |
|        employee        |  -0.3734057385844801   |
|          week          |  -0.3682880866103344   |
|        manager         |  -0.3512892283980237   |
|          time          |  -0.2820404402774594   |
|          said          |  -0.2749278854276806   |
|          paid          |   -0.249465112103778   |
|          one           |  -0.22108340514968422  |
|          due           |  -0.22106109604088695  |
|           me           |  -0.21816454218256096  |
|          wage          |  -0.21381773589196387  |
|         worker         |  -0.21125083218339546  |
|         offer          |  -0.21105603153393127  |
|          sick          |  -0.21082803488072197  |
|         of

In [ ]:
with open('count_basic_transformer_all.csv', 'w', newline='') as f_output:
    f_output.write(importantfeature_basic_transformer_all.get_csv_string())
with open('tfidf_basic_transformer_all.csv', 'w', newline='') as f_output:
    f_output.write(importantfeature2_basic_transformer_all.get_csv_string())

## Sentiment Analysis with BERT


In [ ]:
# from https://www.youtube.com/watch?v=Ew72EAgM7FM

In [ ]:
happy_tc = HappyTextClassification(model_type="DISTILBERT", model_name="distilbert-base-uncased-finetuned-sst-2-english", num_labels=2)

In [ ]:
results_BERT=[]

for t in threads_reddit:
  if len(t)<2100:
    result = happy_tc.classify_text(t)
    results_BERT.append({'content':t,'label':result.label,'score':result.score})

print(results_BERT[:3])


[{'content': 'effective immediately rules ', 'label': 'POSITIVE', 'score': 0.9992859959602356}, {'content': 'company car ordered want leave england uk ', 'label': 'NEGATIVE', 'score': 0.9965292811393738}, {'content': 'would violate non compete competitive services defined marketing whole provide seo services philadelphia pa basically give gist situation terminated back november violating contract give background info worked seo associate large marketing agency offered plenty marketing services not seo pennsylvania i go much background employment tried best bring leads could earn referral minimum budget could consult smaller clients looking budget thought would fair game since employer offers integrated marketing services one service much higher budget requirement so here question comes in violating non compete continue provide seo services i terminated severance agreement states non compete still effect would still apply seo specifically since provide full marketing services seo here s

In [ ]:
df_BERT = pd.DataFrame(results_BERT)
df_BERT.head()

,content,label,score
0,effective immediately rules,POSITIVE,0.999286
1,company car ordered want leave england uk,NEGATIVE,0.996529
2,would violate non compete competitive services...,NEGATIVE,0.997682
3,ontario can us company located north carolina ...,NEGATIVE,0.997027
4,california boss pressured drugs drugs company ...,NEGATIVE,0.997496


In [ ]:
df_BERT.count()

content    971
label      971
score      971
dtype: int64

In [ ]:
df_BERT['label_value']=0
df_BERT.loc[df_BERT['label'] == "POSITIVE",'label_value'] = 1
df_BERT.loc[df_BERT['label'] == "NEGATIVE", 'label_value'] = -1
df_BERT.head()

,content,label,score,label-value,label_value
0,effective immediately rules,POSITIVE,0.999286,0,1
1,company car ordered want leave england uk,NEGATIVE,0.996529,0,-1
2,would violate non compete competitive services...,NEGATIVE,0.997682,0,-1
3,ontario can us company located north carolina ...,NEGATIVE,0.997027,0,-1
4,california boss pressured drugs drugs company ...,NEGATIVE,0.997496,0,-1


In [ ]:
df_BERT_stripped=df_BERT[['content','label_value','label']]
df_BERT_stripped.count()


content        971
label_value    971
label          971
dtype: int64

In [ ]:
df_BERT_stripped.to_csv('sentiment_BERT.csv',encoding='utf-8',index=False)
df_BERT_stripped.label.value_counts()

NEGATIVE    952
POSITIVE     19
Name: label, dtype: int64

In [102]:
train_BERT,test_BERT=train_test_split(df_BERT_stripped,test_size=.3,random_state=42,shuffle=True)

print(len(train_BERT))
print(len(test_BERT))
print(train_BERT.columns)

679
292
Index(['content', 'label_value', 'label'], dtype='object')


In [103]:
#unigram

countvect_BERT = CountVectorizer(analyzer = "word",tokenizer = LemmaTokenizer(), ngram_range=(1,1),min_df=10,max_features=500)
tfidfvect_BERT = TfidfVectorizer(analyzer = "word",tokenizer = LemmaTokenizer(), ngram_range=(1,1),min_df=10,max_features=500)
x_train_count_BERT=countvect_BERT.fit_transform(train_BERT['content']).toarray()
x_test_count_BERT = countvect_BERT.transform(test_BERT['content']).toarray()
x_train_tfidf_BERT= tfidfvect_BERT.fit_transform(train_BERT['content']).toarray()
x_test_tfidf_BERT = tfidfvect_BERT.transform(test_BERT['content']).toarray()
y_train_BERT = train_BERT['label_value']
y_test_BERT = test_BERT['label_value']
lgr=LogisticRegression()
lgr.fit(x_train_count_BERT,y_train_BERT)
lgr.score(x_test_count_BERT,y_test_BERT)
lgr.coef_[0]
i=0
importantfeature_BERT = PrettyTable(["Feature","Score"])
for feature, importance in zip(countvect_BERT.get_feature_names_out(),lgr.coef_[0]):
    importantfeature_BERT.add_row([feature,importance])
importantfeature_BERT.sortby = "Score"

print(importantfeature_BERT)

+----------------+------------------------+
|    Feature     |         Score          |
+----------------+------------------------+
|       i        |  -0.6982445202725349   |
|     would      |  -0.4913397407962228   |
|   employment   |  -0.4465773013365987   |
|    employee    |  -0.4028863321776362   |
|      said      |  -0.37194953604197106  |
|    manager     |  -0.37006647789179237  |
|      hour      |  -0.3653472554474224   |
|      week      |  -0.36276138955936954  |
|    working     |  -0.3621219226873303   |
|      due       |  -0.35249299177316507  |
|     state      |  -0.34840463287306545  |
|     worker     |  -0.3417517599006436   |
|      pay       |  -0.31748412024981904  |
|     asking     |  -0.3171287006590025   |
|  information   |  -0.3089174673901681   |
|    recently    |  -0.30462666651243997  |
|     claim      |  -0.29926460978914027  |
|      time      |  -0.2980587010123432   |
|      one       |  -0.29326347014916154  |
|     shift      |  -0.292739047

In [104]:
lgr=LogisticRegression()
lgr.fit(x_train_tfidf_BERT,y_train_BERT)
lgr.score(x_test_tfidf_BERT,y_test_BERT)
lgr.coef_[0]

importantfeature2_BERT = PrettyTable(["Feature","Score"])
for feature, importance in zip(tfidfvect_BERT.get_feature_names_out(),lgr.coef_[0]):
    importantfeature2_BERT.add_row([feature,importance])
importantfeature2_BERT.sortby = "Score"

print(importantfeature2_BERT)

+----------------+------------------------+
|    Feature     |         Score          |
+----------------+------------------------+
|       i        |  -0.5052264155064912   |
|      hour      |  -0.5034134533224666   |
|     would      |  -0.4358663867142656   |
|    employee    |  -0.37337141958140524  |
|      week      |  -0.3677118592053136   |
|    manager     |  -0.34936245566084434  |
|      time      |  -0.2816178255574681   |
|      said      |  -0.27314084638812725  |
|      paid      |  -0.25138527203685884  |
|      told      |  -0.24806467672394655  |
|      one       |  -0.21976677020191995  |
|      due       |  -0.21919978816097369  |
|       me       |  -0.21656582157188295  |
|      sick      |  -0.21563064613863464  |
|      wage      |  -0.21328119177730945  |
|     worker     |  -0.21270165505022234  |
|     offer      |  -0.2116215638913993   |
|     office     |  -0.2099737029427191   |
|    working     |  -0.2099214743367451   |
|    vacation    |  -0.208749216

In [105]:
with open('count_unigaram_BERT.csv', 'w', newline='') as f_output:
    f_output.write(importantfeature_BERT.get_csv_string())
with open('tfidf_unigram_BERT.csv', 'w', newline='') as f_output:
    f_output.write(importantfeature2_BERT.get_csv_string())

In [106]:
#bigram

countvect_BERT_2 = CountVectorizer(analyzer = "word",tokenizer = LemmaTokenizer(), ngram_range=(2,2),min_df=10,max_features=500)
tfidfvect_BERT_2 = TfidfVectorizer(analyzer = "word",tokenizer = LemmaTokenizer(), ngram_range=(2,2),min_df=10,max_features=500)
x_train_count_BERT_2=countvect_BERT_2.fit_transform(train_BERT['content']).toarray()
x_test_count_BERT_2 = countvect_BERT_2.transform(test_BERT['content']).toarray()
x_train_tfidf_BERT_2= tfidfvect_BERT_2.fit_transform(train_BERT['content']).toarray()
x_test_tfidf_BERT_2 = tfidfvect_BERT_2.transform(test_BERT['content']).toarray()
y_train_BERT_2 = train_BERT['label_value']
y_test_BERT_2 = test_BERT['label_value']
lgr=LogisticRegression()
lgr.fit(x_train_count_BERT_2,y_train_BERT_2)
lgr.score(x_test_count_BERT_2,y_test_BERT_2)
lgr.coef_[0]
i=0
importantfeature_BERT_2 = PrettyTable(["Feature","Score"])
for feature, importance in zip(countvect_BERT_2.get_feature_names_out(),lgr.coef_[0]):
    importantfeature_BERT_2.add_row([feature,importance])
importantfeature_BERT_2.sortby = "Score"

print(importantfeature_BERT_2)

+------------------------+----------------------+
|        Feature         |        Score         |
+------------------------+----------------------+
|     employment law     | -0.5597628717029692  |
|        can not         | -0.5538047217564354  |
|         let go         | -0.46982220257984847 |
|       feel like        | -0.3931502263010405  |
|        last day        | -0.3911390212202745  |
|        two week        | -0.37817444774931025 |
|       last week        | -0.37145758602441814 |
|        get paid        | -0.36221136620148126 |
|       part time        | -0.35461073684658256 |
|        year ago        | -0.3422961316986528  |
|       told would       | -0.31424851152473077 |
|     vacation time      | -0.31119837174919296 |
|      even though       | -0.3111195452856148  |
|        laid off        | -0.3090902405691372  |
|       sick leave       | -0.30513327245320115 |
|      my question       | -0.3003751571893883  |
|        new job         | -0.29405424289414916 |


In [107]:
lgr=LogisticRegression()
lgr.fit(x_train_tfidf_BERT_2,y_train_BERT_2)
lgr.score(x_test_tfidf_BERT_2,y_test_BERT_2)
lgr.coef_[0]

importantfeature2_BERT_2 = PrettyTable(["Feature","Score"])
for feature, importance in zip(tfidfvect_BERT_2.get_feature_names_out(),lgr.coef_[0]):
    importantfeature2_BERT_2.add_row([feature,importance])
importantfeature2_BERT_2.sortby = "Score"

print(importantfeature2_BERT_2)

+------------------------+----------------------+
|        Feature         |        Score         |
+------------------------+----------------------+
|     employment law     | -0.31598592677431603 |
|        can not         |  -0.310595291415577  |
|         let go         | -0.28184917524248787 |
|       feel like        | -0.24035834490208102 |
|        get paid        | -0.22426686767648707 |
|       part time        | -0.21048669788241497 |
|        year ago        | -0.20965181568999958 |
|        two week        | -0.2061585696876413  |
|     vacation time      | -0.2016398220402666  |
|      even though       | -0.19965276977547067 |
|       last week        | -0.19675882982219536 |
|  employment contract   | -0.19229662109862364 |
|        last day        | -0.1865879626916507  |
|       last year        | -0.18466110132373936 |
| independent contractor | -0.1808711977362416  |
|      minimum wage      | -0.17999357214849174 |
|         i told         | -0.1792564253020752  |


In [108]:
with open('count_bigram_BERT.csv', 'w', newline='') as f_output:
    f_output.write(importantfeature_BERT_2.get_csv_string())
with open('tfidf_bigram_BERT.csv', 'w', newline='') as f_output:
    f_output.write(importantfeature2_BERT_2.get_csv_string())

In [109]:
#trigram

countvect_BERT_3 = CountVectorizer(analyzer = "word",tokenizer = LemmaTokenizer(), ngram_range=(3,3),min_df=10,max_features=500)
tfidfvect_BERT_3 = TfidfVectorizer(analyzer = "word",tokenizer = LemmaTokenizer(), ngram_range=(3,3),min_df=10,max_features=500)
x_train_count_BERT_3=countvect_BERT_3.fit_transform(train_BERT['content']).toarray()
x_test_count_BERT_3 = countvect_BERT_3.transform(test_BERT['content']).toarray()
x_train_tfidf_BERT_3= tfidfvect_BERT_3.fit_transform(train_BERT['content']).toarray()
x_test_tfidf_BERT_3 = tfidfvect_BERT_3.transform(test_BERT['content']).toarray()
y_train_BERT_3 = train_BERT['label_value']
y_test_BERT_3 = test_BERT['label_value']
lgr=LogisticRegression()
lgr.fit(x_train_count_BERT_3,y_train_BERT_3)
lgr.score(x_test_count_BERT_3,y_test_BERT_3)
lgr.coef_[0]
i=0
importantfeature_BERT_3 = PrettyTable(["Feature","Score"])
for feature, importance in zip(countvect_BERT_3.get_feature_names_out(),lgr.coef_[0]):
    importantfeature_BERT_3.add_row([feature,importance])
importantfeature_BERT_3.sortby = "Score"

print(importantfeature_BERT_3)

+------------------+----------------------+
|     Feature      |        Score         |
+------------------+----------------------+
| long story short | -0.21550616828479421 |
|  my question is  | -0.2011424627578996  |
+------------------+----------------------+


In [110]:
lgr=LogisticRegression()
lgr.fit(x_train_tfidf_BERT_3,y_train_BERT_3)
lgr.score(x_test_tfidf_BERT_3,y_test_BERT_3)
lgr.coef_[0]

importantfeature2_BERT_3 = PrettyTable(["Feature","Score"])
for feature, importance in zip(tfidfvect_BERT_3.get_feature_names_out(),lgr.coef_[0]):
    importantfeature2_BERT_3.add_row([feature,importance])
importantfeature2_BERT_3.sortby = "Score"

print(importantfeature2_BERT_3)

+------------------+----------------------+
|     Feature      |        Score         |
+------------------+----------------------+
| long story short | -0.21310220258160853 |
|  my question is  | -0.19885275808548722 |
+------------------+----------------------+


In [111]:
with open('count_traigram_BERT.csv', 'w', newline='') as f_output:
    f_output.write(importantfeature_BERT_3.get_csv_string())
with open('tfidf_trigram_BERT.csv', 'w', newline='') as f_output:
    f_output.write(importantfeature2_BERT_3.get_csv_string())

In [112]:
#all

countvect_BERT_all = CountVectorizer(analyzer = "word",tokenizer = LemmaTokenizer(), ngram_range=(1,3),min_df=10,max_features=500)
tfidfvect_BERT_all = TfidfVectorizer(analyzer = "word",tokenizer = LemmaTokenizer(), ngram_range=(1,3),min_df=10,max_features=500)
x_train_count_BERT_all=countvect_BERT_all.fit_transform(train_BERT['content']).toarray()
x_test_count_BERT_all = countvect_BERT_all.transform(test_BERT['content']).toarray()
x_train_tfidf_BERT_all= tfidfvect_BERT_all.fit_transform(train_BERT['content']).toarray()
x_test_tfidf_BERT_all = tfidfvect_BERT_all.transform(test_BERT['content']).toarray()
y_train_BERT_all = train_BERT['label_value']
y_test_BERT_all = test_BERT['label_value']
lgr=LogisticRegression()
lgr.fit(x_train_count_BERT_all,y_train_BERT_all)
lgr.score(x_test_count_BERT_all,y_test_BERT_all)
lgr.coef_[0]
i=0
importantfeature_BERT_all = PrettyTable(["Feature","Score"])
for feature, importance in zip(countvect_BERT_all.get_feature_names_out(),lgr.coef_[0]):
    importantfeature_BERT_all.add_row([feature,importance])
importantfeature_BERT_all.sortby = "Score"

print(importantfeature_BERT_all)

+------------------------+------------------------+
|        Feature         |         Score          |
+------------------------+------------------------+
|           i            |  -0.7024530046373825   |
|         would          |  -0.5119241261359622   |
|       employment       |  -0.4347335329783244   |
|        employee        |  -0.43283427317529594  |
|          said          |  -0.38730382827511006  |
|          due           |   -0.382598412242826   |
|        manager         |  -0.3818413242117749   |
|          hour          |  -0.3743663726562657   |
|          week          |  -0.3699234552117881   |
|         state          |  -0.3638660126701316   |
|        working         |  -0.35849044496052934  |
|         worker         |  -0.3501802370101665   |
|          time          |  -0.3471991448485691   |
|          pay           |  -0.32817577383107266  |
|         claim          |  -0.3238057385704046   |
|          one           |  -0.3202501092399268   |
|      infor

In [113]:
lgr=LogisticRegression()
lgr.fit(x_train_tfidf_BERT_all,y_train_BERT_all)
lgr.score(x_test_tfidf_BERT_all,y_test_BERT_all)
lgr.coef_[0]

importantfeature2_BERT_all = PrettyTable(["Feature","Score"])
for feature, importance in zip(tfidfvect_BERT_all.get_feature_names_out(),lgr.coef_[0]):
    importantfeature2_BERT_all.add_row([feature,importance])
importantfeature2_BERT_all.sortby = "Score"

print(importantfeature2_BERT_all)

+------------------------+------------------------+
|        Feature         |         Score          |
+------------------------+------------------------+
|           i            |  -0.5060531665986403   |
|          hour          |  -0.5046733937165042   |
|         would          |  -0.43507803016217195  |
|        employee        |  -0.3734057385844801   |
|          week          |  -0.3682880866103344   |
|        manager         |  -0.3512892283980237   |
|          time          |  -0.2820404402774594   |
|          said          |  -0.2749278854276806   |
|          paid          |   -0.249465112103778   |
|          one           |  -0.22108340514968422  |
|          due           |  -0.22106109604088695  |
|           me           |  -0.21816454218256096  |
|          wage          |  -0.21381773589196387  |
|         worker         |  -0.21125083218339546  |
|         offer          |  -0.21105603153393127  |
|          sick          |  -0.21082803488072197  |
|         of

In [114]:
with open('count_BERT_all.csv', 'w', newline='') as f_output:
    f_output.write(importantfeature_BERT_all.get_csv_string())
with open('tfidf_BERT_all.csv', 'w', newline='') as f_output:
    f_output.write(importantfeature2_BERT_all.get_csv_string())

In [115]:
!zip -r /content/file.zip /content

  adding: content/ (stored 0%)
  adding: content/.config/ (stored 0%)
  adding: content/.config/.last_survey_prompt.yaml (stored 0%)
  adding: content/.config/gce (stored 0%)
  adding: content/.config/config_sentinel (stored 0%)
  adding: content/.config/.last_update_check.json (deflated 22%)
  adding: content/.config/.last_opt_in_prompt.yaml (stored 0%)
  adding: content/.config/default_configs.db (deflated 98%)
  adding: content/.config/configurations/ (stored 0%)
  adding: content/.config/configurations/config_default (deflated 15%)
  adding: content/.config/active_config (stored 0%)
  adding: content/.config/logs/ (stored 0%)
  adding: content/.config/logs/2023.12.05/ (stored 0%)
  adding: content/.config/logs/2023.12.05/14.18.28.146801.log (deflated 91%)
  adding: content/.config/logs/2023.12.05/14.24.19.217285.log (deflated 58%)
  adding: content/.config/logs/2023.12.05/14.24.27.916132.log (deflated 57%)
  adding: content/.config/logs/2023.12.05/14.24.28.650298.log (deflated 57%)

### Comparing files

In [ ]:
df1 = pd.read_csv('tfidf_bigram_basic_transformer.csv')
df2 = pd.read_csv('tfidf_bigram_BERT.csv')
df3 = pd.read_csv('tfidf_biigrams_NLTK.csv')
common_phrases = pd.merge(df1, df2, on='Feature', how='inner').merge(df3, on='Feature', how='inner')['Feature']
print("Common Phrases:", common_phrases.tolist())

Common Phrases: ['employment law', 'can not', 'let go', 'feel like', 'get paid', 'part time', 'year ago', 'two week', 'vacation time', 'even though', 'last week', 'employment contract', 'last day', 'last year', 'independent contractor', 'minimum wage', 'i told', 'new job', 'told would', 'non compete', 'laid off', 'i work', 'pay period', 'new york', 'hi all', 'sick leave', 'my question', 'day week', 'work hour', 'next day', 'company year', 'first time', 'week ago', 'dont want', 'would like', 'hour worked', 'time employee', 'hour week', 'wrongful termination', 'fast forward', 'co worker', 'question is', 'i worked', 'pay back', 'company work', 'dont know', 'let know', 'manager told', 'make sure', 'i sure', 'sick time', 'next week', 'offer letter', 'ontario canada', 'employment lawyer', 'i recently', 'every week', 'work company', 'seems like', 'said would', 'working hour', 'greatly appreciated', 'day later', 'paid hour', 'hour day', 'lunch break', 'i wondering', 'thank advance', 'long story', 'i looking', 'work home', 'company policy', 'hello i', 'i also', 'manager said', 'i want', 'per week', 'hour work', 'i going', 'one day', 'story short', 'week notice', 'day work', 'start date', 'ive never', 'work week', 'first week', 'would appreciated', 'current job', 'work day', 'new position', 'work remotely', 'i working', 'thank you', 'mental health', 'i know', 'last month', 'come back', 'paid time', 'getting paid', 'return work', 'legal action', 'hi everyone', 'is legal', 'would need', 'month ago', 'full time', 'thanks advance']